In [2]:
# Import library

from resnet152_lib import *


Using device: cuda


In [3]:
# Values for z-score norm

data_mean = (0.4180, 0.3998, 0.3396)
data_std = (0.2728, 0.2571, 0.2389)

# Validation size

validation_size = 0.0
seed_for_split = 42

In [4]:
# Hiper-params

lr_value = 0.0001
lr_decay = 0.7
epochs = 30
dropout_value = 0.1
hidden_layer = 2048
batch_size = 16
network_type = "resnet152"

In [5]:
# Paths

train_data_path = "./input/train_data.csv"
train_img_path = "./input/train_img"
sample_sub_path = "./input/sample_submission.csv"
test_img_path = "./input/test_img/"
sub_dst_path = "submission-resnet152.csv"

In [6]:
# Getting the data

futuramaDataset = FuturamaDataset(train_data_path, train_img_path)

In [7]:
# Split into train and val

train_len = int(futuramaDataset.__len__())
test_len = futuramaDataset.__len__() - train_len
train_set, val_set = torch.utils.data.random_split(
    futuramaDataset,
    [train_len, test_len],
    generator=torch.Generator().manual_seed(seed_for_split),
)

random_transforms = [
    transforms.RandomAffine(degrees=(0, 360), scale=(0.25, 0.3)),
    transforms.ColorJitter(hue=(-0.5, 0.5)),
    transforms.RandomErasing(p=1.0, scale=(0.2, 0.5), value="random"),
]


train_set.dataset.transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.AutoAugment(),
        transforms.RandomChoice(random_transforms),
        transforms.Normalize(data_mean, data_std),
    ]
)

val_set.dataset.transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(data_mean, data_std),
    ]
)


In [8]:
# Create the dataloader for train and test

train_loader = DataLoader(
    train_set, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True
)
val_loader = DataLoader(
    val_set, batch_size=batch_size, shuffle=False, num_workers=8, drop_last=False
)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [9]:
# Download ResNet152 pretrained with Imaginet_V2

model = FuturamaResnet(network_type, hidden_layer, dropout_value).to(device)

using resnet152


Downloading: "https://download.pytorch.org/models/resnet152-f82ba261.pth" to /root/.cache/torch/hub/checkpoints/resnet152-f82ba261.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

In [10]:
# Training

training(model, device, None, lr_value, lr_decay, epochs, train_loader, val_loader)

Adjusting learning rate of group 0 to 1.0000e-04.
Epoch Train[1/30], Step [509/509], Loss: 0.4431
Adjusting learning rate of group 0 to 7.0000e-05.
Epoch Train[2/30], Step [509/509], Loss: 0.0827
Adjusting learning rate of group 0 to 4.9000e-05.
Epoch Train[3/30], Step [509/509], Loss: 0.0269
Adjusting learning rate of group 0 to 3.4300e-05.
Epoch Train[4/30], Step [509/509], Loss: 0.0127
Adjusting learning rate of group 0 to 2.4010e-05.
Epoch Train[5/30], Step [509/509], Loss: 0.0074
Adjusting learning rate of group 0 to 1.6807e-05.
Epoch Train[6/30], Step [509/509], Loss: 0.0036
Adjusting learning rate of group 0 to 1.1765e-05.
Epoch Train[7/30], Step [509/509], Loss: 0.0028
Adjusting learning rate of group 0 to 8.2354e-06.
Epoch Train[8/30], Step [509/509], Loss: 0.0031
Adjusting learning rate of group 0 to 5.7648e-06.
Epoch Train[9/30], Step [509/509], Loss: 0.0024
Adjusting learning rate of group 0 to 4.0354e-06.
Epoch Train[10/30], Step [509/509], Loss: 0.0015
Adjusting learning 

In [11]:
# Create a submission using the trained model

create_submission(
    model, sample_sub_path, test_img_path, sub_dst_path, data_mean, data_std)